In [1]:
import os

In [ ]:
#command that shows the current working directory
%pwd

'c:\\Users\\Priya\\NLP\\Text-Summarizer-Project\\research'

In [3]:
#changes the current working directory to the parent directory
os.chdir("../")
#The current working directory becomes: 'c:\\Users\\Priya\\NLP\\Text-Summarizer-Project'

In [4]:
%pwd

'c:\\Users\\Priya\\NLP\\Text-Summarizer-Project'

In [ ]:
#This line imports the dataclass decorator from Python’s built-in dataclasses module
#A dataclass is a Python feature that automatically creates boilerplate code (like __init__, __repr__, and __eq__ methods) for classes that are mainly used to store data.
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
#Defines a new data structure (class) to hold all config values related to the data ingestion phase
#DataIngestionConfig is an entity — specifically, a configuration entity.
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
pip install --upgrade ensure

Note: you may need to restart the kernel to use updated packages.


In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [ ]:
#class ConfigurationManager, Read YAML config files, Create necessary directories, Generate configuration objects (like DataIngestionConfig) for various pipeline step
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        #Reads the contents of config.yaml using a helper function read_yaml() and stores it as a Python dictionary
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    #This method is designed to construct and return a DataIngestionConfig object — that is, an object holding the config values specific to data ingestion.
    #The return type is declared as DataIngestionConfig
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        #Grabs the data_ingestion section from config.yaml
        config = self.config.data_ingestion
        #creates the data_ingestion folder inside the artifacts/ directory
        create_directories([config.root_dir])
        #Create an instance of the DataIngestionConfig class and Populate it with values from the YAML file (via config).
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        #returns the data_ingestion_config object
        return data_ingestion_config
    

In [9]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [ ]:
class DataIngestion:
    #It expects an argument config, which must be an instance of DataIngestionConfig
    def __init__(self, config: DataIngestionConfig):
        #Stores the config object as an instance variable so other methods can use it.
        self.config = config

    def download_file(self):
        #Checks whether the data.zip file already exists locally, If the file does not exist, download it
        #Downloads the file from source_URL to local_data_file.
        #Returns the filename and HTTP response headers
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        #zipfile.ZipFile(...) opens the downloaded ZIP file in read mode.
        #zip_ref.extractall(...) extracts all files inside to the unzip_path directory
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:

print("Config file exists:", config_path.exists())
print("Params file exists:", params_path.exists())

Config file exists: True
Params file exists: True


In [12]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\Users\Priya\NLP\Text-Summarizer-Project


In [ ]:
try:
    #creates an object of the ConfigurationManager class
    #config is an object of ConfigurationManager
    config = ConfigurationManager()
    #Calls the method get_data_ingestion_config() on the config object
    #data_ingestion_config  is an object of type DataIngestionConfig
    data_ingestion_config = config.get_data_ingestion_config()
    #Creates a new object of the DataIngestion class.
    #It receives the config object (DataIngestionConfig) and stores it in self.config.
    #passing the value of data_ingestion_config to the config parameter of the constructor.
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-09 17:33:42,103: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-09 17:33:42,104: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-09 17:33:42,104: INFO: common: Created directory at: artifacts]
[2025-06-09 17:33:42,104: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-06-09 17:33:42,104: INFO: 4169478213: File already exists of size: ~ 7718 KB]
